# Libraries

In [1]:
from scipy.sparse import csc_matrix, csr_matrix
import numpy as np
from sklearn.random_projection import SparseRandomProjection
from scipy.spatial.distance import cosine
from collections import defaultdict
from tqdm import tqdm

# Load Data

In [2]:

# Load the data
data = np.load('user_movie_rating.npy')

In [3]:
data

array([[     1,     30,      3],
       [     1,    157,      3],
       [     1,    173,      4],
       ...,
       [103703,  17622,      2],
       [103703,  17627,      4],
       [103703,  17764,      4]], dtype=int64)

### Create CSR matrix to save memory

In [27]:
# Check if the dataset uses 1-based indexing and convert to 0-based if necessary
if np.min(data[:, 0]) == 1:
    data[:, 0] -= 1  # Convert user_id to 0-based index
if np.min(data[:, 1]) == 1:
    data[:, 1] -= 1  # Convert movie_id to 0-based index

# Create the sparse matrix
num_users = np.max(data[:, 0]) + 1
num_movies = np.max(data[:, 1]) + 1
ratings_matrix = csr_matrix((data[:, 2], (data[:, 0], data[:, 1])), shape=(num_users, num_movies))

# Create Sets of Rated Movies for Each User
user_movie_sets = defaultdict(set)
for user_id, movie_id, _ in data: 
    user_movie_sets[user_id].add(movie_id)

In [49]:
# Check NumPy array computation load
print("NumPy Array 'sampled_data' Size:")
print("Shape:", sampled_data.shape)
print("Total Elements:", sampled_data.size)
print("Bytes Consumed:", sampled_data.nbytes)

NumPy Array 'sampled_data' Size:
Shape: (65225506, 3)
Total Elements: 195676518
Bytes Consumed: 1565412144


In [50]:
# Check CSR matrix computation load
print("\nSciPy CSR Matrix 'ratings_matrix' Size:")
print("Shape:", ratings_matrix.shape)
print("Non-zero Elements:", ratings_matrix.nnz)
print("Bytes for Non-zero Elements:", ratings_matrix.data.nbytes)
print("Total Bytes (including indices and indptr):",
      ratings_matrix.data.nbytes + ratings_matrix.indices.nbytes + ratings_matrix.indptr.nbytes)


SciPy CSR Matrix 'ratings_matrix' Size:
Shape: (103703, 17770)
Non-zero Elements: 65225506
Bytes for Non-zero Elements: 521804048
Total Bytes (including indices and indptr): 783120888


CSR Matrix seems to use almost half as much bytes as the NumPy array.

# Individual components to get a feeling for the mechanism (algorithms for different similarty measure can be found below)

### Minhashing Jaccard Similarity

In [30]:

# Convert to CSC format for efficient column operations
ratings_matrix_csc = csc_matrix(ratings_matrix)

def create_minhash_signature_with_permutations(column, permutations):
    """
    Create a minhash signature for a given column of the matrix using row permutations.
    """
    signature = []
    for perm in permutations:
        # Find the index of the first non-zero element in the permuted order
        permuted_indices = perm[column.nonzero()[0]]
        min_index = np.min(permuted_indices) if permuted_indices.size > 0 else np.inf
        signature.append(min_index)
    return signature

# Number of permutations (signature length)
num_permutations = 150  

# Generate random permutations
np.random.seed(42)  # For reproducibility
num_rows = ratings_matrix_csc.shape[0]
permutations = [np.random.permutation(num_rows) for _ in range(num_permutations)]

# Create Minhash signatures for each user with a progress bar
minhash_signatures = {}
for user_id in tqdm(range(ratings_matrix_csc.shape[1]), desc="Computing Minhash Signatures"):
    user_column = ratings_matrix_csc[:, user_id]
    minhash_signatures[user_id] = create_minhash_signature_with_permutations(user_column, permutations)


Computing Minhash Signatures: 100%|██████████| 17770/17770 [01:39<00:00, 178.18it/s]


In [31]:
minhash_signatures

{0: [119,
  85,
  24,
  18,
  457,
  48,
  5,
  261,
  234,
  234,
  114,
  477,
  414,
  263,
  48,
  168,
  427,
  134,
  370,
  1087,
  126,
  25,
  42,
  12,
  334,
  350,
  319,
  1536,
  347,
  61,
  620,
  47,
  1112,
  40,
  70,
  103,
  23,
  439,
  348,
  260,
  964,
  728,
  651,
  555,
  615,
  199,
  484,
  224,
  212,
  220,
  143,
  376,
  297,
  456,
  536,
  70,
  240,
  366,
  410,
  3,
  26,
  117,
  442,
  226,
  538,
  31,
  519,
  435,
  48,
  93,
  78,
  843,
  189,
  536,
  227,
  921,
  292,
  392,
  330,
  89,
  329,
  299,
  133,
  546,
  85,
  90,
  139,
  43,
  506,
  231,
  1193,
  317,
  142,
  624,
  110,
  473,
  154,
  267,
  523,
  300],
 1: [300,
  1087,
  641,
  2445,
  631,
  706,
  607,
  132,
  1854,
  4075,
  1656,
  3854,
  830,
  1141,
  1381,
  7662,
  5463,
  597,
  992,
  2614,
  1756,
  4748,
  130,
  4928,
  34,
  1227,
  1477,
  853,
  2194,
  15522,
  1575,
  513,
  1527,
  4160,
  190,
  876,
  1595,
  1566,
  1416,
  811,
  180,
  375


##### Context for Understanding Minhash Signatures
Basic Concept of Minhashing:

Minhashing is a technique used in the context of large sets to efficiently estimate their Jaccard similarity (the similarity between two sets is the size of their intersection divided by the size of their union).
In the context of your Netflix dataset, each user can be represented as a set of movies they have rated. The Jaccard similarity between two users is a measure of how similar their movie preferences are.

Minhash Signature:

A Minhash signature is a compressed representation of this user's set (movie preferences).
Each element in a Minhash signature is the minimum hash value (or in your case, the first non-zero element in a randomly permuted order) obtained from applying a hash function (or permutation) to the set.
The length of the signature is determined by the number of hash functions (or permutations) you use. In your case, it's between 80-150.
These signatures have a special property: The probability that two signatures agree in a particular place is equal to the Jaccard similarity of the sets they represent.

minhash_signatures Dictionary:

In your implementation, minhash_signatures is a dictionary where each key is a user ID, and the corresponding value is the Minhash signature of that user.
The Minhash signature for each user is a list (or array) of values, each derived from one of the permutations applied to the user’s set of rated movies.
The length of each user's Minhash signature in this dictionary is the same, determined by the number of permutations you used

##### Practical Use in LSH

Efficient Similarity Estimation:

By comparing these signatures, you can efficiently estimate the Jaccard similarity between any two users. This is much quicker than directly comparing the sets of movies each user has rated, especially as the number of movies grows large.

LSH for Candidate Pair Selection:

In LSH, these Minhash signatures are used to quickly find candidate pairs of users who might have high similarity.
The LSH technique hashes users into buckets such that similar users (based on their Minhash signatures) are likely to end up in the same bucket.
This process significantly reduces the number of user pairs that need to be checked for similarity, as you only compare users within the same buckets.

In summary, minhash_signatures in your implementation is a compact representation of users' movie preferences, enabling efficient similarity calculations. These signatures are the cornerstone of applying LSH to identify users with similar movie-rating patterns in a computationally efficient manner.

### LSH Jaccard Similarity

In [37]:
def hash_band(band):
    return hashlib.sha1(str(band).encode()).hexdigest()

# LSH Parameters
num_bands = 6  # Example value, adjust as needed
rows_per_band = 15  # Adjust as needed (last band might have more rows)

# Hashing bands of signatures into buckets
buckets = defaultdict(list)
for user_id, signature in tqdm(minhash_signatures.items(), desc="Hashing bands to buckets"):
    for band_idx in range(num_bands):
        start_idx = band_idx * rows_per_band
        end_idx = (start_idx + rows_per_band) if (band_idx < num_bands - 1) else len(signature)  # Adjust for last band
        band = tuple(signature[start_idx:end_idx])
        bucket_key = hash_band(band)
        buckets[bucket_key].append(user_id)

# Generating candidate pairs with bucket size check and excluding self-pairings
candidate_pairs = set()
max_bucket_size = 50  # Example value to limit large buckets, adjust as needed
for bucket_users in tqdm(buckets.values(), desc="Generating candidate pairs"):
    if len(bucket_users) > 1 and len(bucket_users) <= max_bucket_size:
        for i in range(len(bucket_users)):
            for j in range(i + 1, len(bucket_users)):
                if bucket_users[i] != bucket_users[j]:  # Exclude self-pairings
                    candidate_pairs.add((bucket_users[i], bucket_users[j]))


Generating candidate pairs: 100%|██████████| 106597/106597 [00:00<00:00, 2411452.77it/s]


In [38]:
len(candidate_pairs)

31

### Calculate similarites 

In [34]:
# Create Sets of Rated Movies for Each User
user_movie_sets = defaultdict(set)
for user_id, movie_id, _ in sampled_data:  # Assuming 'sampled_data' contains your data
    user_movie_sets[user_id].add(movie_id)


# Compute Jaccard Similarity for Candidate Pairs
def jaccard_similarity(set1, set2):
    """ Compute the Jaccard Similarity between two sets """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Calculate Jaccard Similarity for each candidate pair
jaccard_similarities = {}
for user1, user2 in candidate_pairs:
    similarity = jaccard_similarity(user_movie_sets[user1], user_movie_sets[user2])
    jaccard_similarities[(user1, user2)] = similarity


# Filter Pairs Based on Jaccard Similarity Threshold
threshold = 0.5
filtered_pairs = {pair: sim for pair, sim in jaccard_similarities.items() if sim > threshold}


In [35]:
jaccard_similarities

{(6509, 17156): 0.1620762711864407,
 (3937, 13762): 0.14642451759364358,
 (3961, 13762): 0.12608695652173912,
 (6973, 17156): 0.21241830065359477,
 (14239, 14690): 0.17893660531697342,
 (6286, 7616): 0.16994106090373282,
 (11063, 14409): 0.1790073230268511,
 (11520, 14239): 0.2291950886766712,
 (13762, 15106): 0.13822894168466524,
 (6036, 11336): 0.180998613037448,
 (3937, 15106): 0.1674641148325359,
 (11282, 14312): 0.1992619926199262,
 (1904, 11063): 0.2631578947368421,
 (3961, 15106): 0.12514220705346984,
 (570, 11063): 0.10130391173520562,
 (11520, 14690): 0.16136363636363638,
 (5925, 11063): 0.23863636363636365,
 (3937, 3961): 0.12531969309462915,
 (5308, 8437): 0.09115281501340483,
 (4305, 11063): 0.21284185493460167,
 (1904, 4305): 0.20578420467185762,
 (11520, 15123): 0.10857908847184987,
 (1904, 1904): 1.0,
 (14549, 15754): 0.0985576923076923,
 (11780, 16953): 0.09009009009009009,
 (570, 5925): 0.118706942236354,
 (570, 11520): 0.14006342494714588,
 (1904, 14409): 0.2587354409

In [36]:
filtered_pairs

{(1904, 1904): 1.0}

#### Summary of Methods for Each Similarity Measure:
Jaccard Similarity: Minhashing + LSH
Cosine Similarity and Discrete Cosine Similarity: Random Projections + LSH

#### Implementation Notes:
Minhashing for JS: Use a set representation of the movies rated by each user. Apply minhashing to these sets to create signatures, and then use LSH to find candidate pairs.
Random Projections for CS and DCS: Transform user rating vectors (or binary vectors for DCS) using random projection matrices. Then apply LSH to find candidate pairs. The hash function for LSH in this case will differ from the one used for Jaccard similarity.

# Jaccard Similarity full function

In [3]:
# Check if the dataset uses 1-based indexing and convert to 0-based if necessary
if np.min(data[:, 0]) == 1:
    data[:, 0] -= 1  # Convert user_id to 0-based index
if np.min(data[:, 1]) == 1:
    data[:, 1] -= 1  # Convert movie_id to 0-based index

# Create the sparse matrix
num_users = np.max(data[:, 0]) + 1
num_movies = np.max(data[:, 1]) + 1
ratings_matrix = csr_matrix((data[:, 2], (data[:, 0], data[:, 1])), shape=(num_users, num_movies))

In [4]:
# Create Sets of Rated Movies for Each User
user_movie_sets = defaultdict(set)
for user_id, movie_id, _ in data: 
    user_movie_sets[user_id].add(movie_id)

In [27]:
def lsh_jaccard_similarity(ratings_matrix, num_permutations=150, num_bands=5, rows_per_band=30, max_bucket_size=50, threshold=0.5):
    
    ### Minhashing
    
    # Convert to CSC format for efficient column operations
    ratings_matrix_csc = csc_matrix(ratings_matrix)

    # Generate random permutations
    np.random.seed(42)  # For reproducibility
    num_rows = ratings_matrix_csc.shape[0]
    permutations = [np.random.permutation(num_rows) for _ in range(num_permutations)]

    # Create Minhash signatures for each user
    minhash_signatures = {}
    for user_id in tqdm(range(ratings_matrix_csc.shape[1]), desc="Computing Minhash Signatures"):
        user_column = ratings_matrix_csc[:, user_id]
        signature = []
        for perm in permutations:
            permuted_indices = perm[user_column.nonzero()[0]]
            min_index = np.min(permuted_indices) if permuted_indices.size > 0 else np.inf
            signature.append(min_index)
        minhash_signatures[user_id] = signature

    ### LSH

    # Hashing bands of signatures into buckets
    buckets = defaultdict(list)
    for user_id, signature in tqdm(minhash_signatures.items(), desc="Hashing bands to buckets"):
        for band_idx in range(num_bands):
            start_idx = band_idx * rows_per_band
            end_idx = start_idx + rows_per_band if band_idx < num_bands - 1 else len(signature)
            band = tuple(signature[start_idx:end_idx])
            bucket_key = hashlib.sha1(str(band).encode()).hexdigest()
            buckets[bucket_key].append(user_id)

    # Generating candidate pairs
    candidate_pairs = set()
    for bucket_users in tqdm(buckets.values(), desc="Generating candidate pairs"):
        if len(bucket_users) > 1 and len(bucket_users) <= max_bucket_size:
            for i in range(len(bucket_users)):
                for j in range(i + 1, len(bucket_users)):
                    if bucket_users[i] != bucket_users[j]:
                        candidate_pairs.add((bucket_users[i], bucket_users[j]))

    ### Calculate similarities

    # Compute Jaccard Similarity for Candidate Pairs
    jaccard_similarities = {}
    for user1, user2 in candidate_pairs:
        set1, set2 = user_movie_sets[user1], user_movie_sets[user2]
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        similarity = intersection / union if union != 0 else 0
        jaccard_similarities[(user1, user2)] = similarity

    # Filter Pairs Based on Jaccard Similarity Threshold
    filtered_pairs = {pair: sim for pair, sim in jaccard_similarities.items() if sim > threshold}

    return filtered_pairs, candidate_pairs, jaccard_similarities



### Parameter tuning to maximize the amount of similar pairs found

In [10]:
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=100, num_bands=6, rows_per_band=15, max_bucket_size=50, threshold=0.5)

print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 106597/106597 [00:00<00:00, 2893047.39it/s]


Number of Candidate pairs: 31
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (2056, 14030) with a similarity of: 0.33779971791255287


In [11]:
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=100, num_bands=10, rows_per_band=10, max_bucket_size=50, threshold=0.5)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 177196/177196 [00:00<00:00, 3581218.67it/s]


Number of Candidate pairs: 1162
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (1541, 7623) with a similarity of: 0.44516594516594515


In [12]:
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=100, num_bands=20, rows_per_band=5, max_bucket_size=50, threshold=0.5)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 341590/341590 [00:00<00:00, 2709018.77it/s]


Number of Candidate pairs: 48592
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (10582, 16792) with a similarity of: 0.4697142857142857


In [13]:
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=100, num_bands=33, rows_per_band=3, max_bucket_size=50, threshold=0.5)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 485018/485018 [00:02<00:00, 208913.56it/s]


Number of Candidate pairs: 342801
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (2740, 8001) with a similarity of: 0.47895652173913045


We observe that the number of candidate pairs and the highest found JS score increase when we increase the number of bands and decrease the rows per band. It does seem like we are getting diminished returns in terms of max JS score, so we will now try some different parameters.

We will now try increasing the max bucket size from 50 to 500, allowing more pairs to be added in the same bucket before we compare them.

In [14]:
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=100, num_bands=33, rows_per_band=3, max_bucket_size=500, threshold=0.5)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 485018/485018 [00:00<00:00, 573116.93it/s]


Number of Candidate pairs: 1106558
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (1323, 10582) with a similarity of: 0.4975429975429975


We observe that computation time increased substantially, as expected, because we need to do more within-bucket comparisons. We see more candidate pairs, and now are just shy of the JS threshold of 0.5.

If we now increase the signature length( num_permutations) we should be crossing the threshhold.

In [15]:
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=150, num_bands=50, rows_per_band=3, max_bucket_size=500, threshold=0.5)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 720391/720391 [00:01<00:00, 518710.83it/s] 


Number of Candidate pairs: 1641443
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (1323, 10582) with a similarity of: 0.4975429975429975


It seems like increasing the signature length increased the number of candidate pairs, but did not increase the max JS score. Considering it also increased the computation time, we will not be using these values.

Let's try increasing the bucket size further, since that yielded considerably higher max JS scores.
We will try an ambitious increase from 500 to 2000. 

In [19]:
# This is the best performing JS model and is thus used for the python file
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=100, num_bands=33, rows_per_band=3, max_bucket_size=2000, threshold=0.5)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 485018/485018 [00:01<00:00, 295554.17it/s]


Number of Candidate pairs: 1302012
Number of Pairs with JS > 0.5: 1
Pair with highest Jaccard similarity: (7623, 10582) with a similarity of: 0.5257472092185812


Above setup yields the best result. We are finally able to find a pair with JS score > 0.5

In [20]:
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=100, num_bands=50, rows_per_band=2, max_bucket_size=2000, threshold=0.5)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 431337/431337 [00:11<00:00, 37881.26it/s] 


Number of Candidate pairs: 10078619
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (1323, 10582) with a similarity of: 0.4975429975429975


Increasing the number of bands further while setting rows per band to 2 yields higher computation time and worse results.

In [23]:
filtered_pairs, candidate_pairs, jaccard_similarities = lsh_jaccard_similarity(
    ratings_matrix, num_permutations=80, num_bands=27, rows_per_band=3, max_bucket_size=50, threshold=0.5)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with JS > 0.5:", len(filtered_pairs))
max_pair, max_similarity = max(jaccard_similarities.items(), key=lambda item: item[1])
print("Pair with highest Jaccard similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 394587/394587 [00:00<00:00, 1460707.25it/s]


Number of Candidate pairs: 292935
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (2740, 8001) with a similarity of: 0.47895652173913045


Keeping r to 3 and decreasing signature length also decreases results.

# 2: Cosine Similarity

In [33]:
def lsh_cosine_similarity(ratings_matrix, num_components=100, num_bands=33, rows_per_band=3, max_bucket_size=50, threshold=0.73):
    # Convert to CSR format for efficient row operations
    ratings_matrix_csr = csr_matrix(ratings_matrix)

    ### Random Projections for LSH

    # Apply random projection to reduce dimensions while preserving cosine similarity
    rp = SparseRandomProjection(n_components=num_components, random_state=42)
    reduced_data = rp.fit_transform(ratings_matrix_csr)

    ### LSH

    # Hashing the reduced data into buckets
    buckets = defaultdict(list)
    for user_id in tqdm(range(reduced_data.shape[0]), desc="Hashing users to buckets"):
        user_vector = reduced_data[user_id, :]
        for band_idx in range(num_bands):
            start_idx = band_idx * rows_per_band
            end_idx = start_idx + rows_per_band if band_idx < num_bands - 1 else num_components
            band = tuple(user_vector[start_idx:end_idx])
            bucket_key = hashlib.sha1(str(band).encode()).hexdigest()
            buckets[bucket_key].append(user_id)

    # Generating candidate pairs
    candidate_pairs = set()
    for bucket_users in tqdm(buckets.values(), desc="Generating candidate pairs"):
        if len(bucket_users) > 1 and len(bucket_users) <= max_bucket_size:
            for i in range(len(bucket_users)):
                for j in range(i + 1, len(bucket_users)):
                    candidate_pairs.add((bucket_users[i], bucket_users[j]))

    ### Calculate Cosine Similarities

    # Compute Cosine Similarity for Candidate Pairs
    cosine_similarities = {}
    for user1, user2 in candidate_pairs:
        # Convert to 1D arrays
        vector1 = reduced_data[user1, :].toarray().ravel()
        vector2 = reduced_data[user2, :].toarray().ravel()

        # Note: 1 - cosine distance is cosine similarity
        similarity = 1 - cosine(vector1, vector2)
        cosine_similarities[(user1, user2)] = similarity

    # Filter Pairs Based on Cosine Similarity Threshold
    filtered_pairs = {pair: sim for pair, sim in cosine_similarities.items() if sim > threshold}

    return filtered_pairs, candidate_pairs, cosine_similarities

In [34]:
filtered_pairs, candidate_pairs, cosine_similarities = lsh_cosine_similarity(
    ratings_matrix, num_components=100, num_bands=33, rows_per_band=3, max_bucket_size=50, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with CS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 37/37 [00:00<?, ?it/s]


Number of Candidate pairs: 1300
Number of Pairs with JS > 0.5: 0
Pair with highest Jaccard similarity: (26587, 65270) with a similarity of: 0.5781013232100951


In [55]:
filtered_pairs, candidate_pairs, cosine_similarities = lsh_cosine_similarity(
    ratings_matrix, num_components=80, num_bands=27, rows_per_band=3, max_bucket_size=500, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with CS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 30/30 [00:00<00:00, 479.99it/s]


Number of Candidate pairs: 145082
Number of Pairs with CS > 0.73: 0
Pair with highest Cosine similarity: (71261, 76656) with a similarity of: 0.7200141068509961


In [58]:
filtered_pairs, candidate_pairs, cosine_similarities = lsh_cosine_similarity(
    ratings_matrix, num_components=60, num_bands=20, rows_per_band=3, max_bucket_size=500, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with CS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 25/25 [00:00<00:00, 421.76it/s]


Number of Candidate pairs: 122861
Number of Pairs with CS > 0.73: 2
Pair with highest Cosine similarity: (42676, 44660) with a similarity of: 0.7619991098716038


In [59]:
filtered_pairs, candidate_pairs, cosine_similarities = lsh_cosine_similarity(
    ratings_matrix, num_components=45, num_bands=15, rows_per_band=3, max_bucket_size=500, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with CS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 21/21 [00:00<00:00, 479.34it/s]


Number of Candidate pairs: 55766
Number of Pairs with CS > 0.73: 5
Pair with highest Cosine similarity: (41747, 56537) with a similarity of: 0.7697812790877132


In [60]:
filtered_pairs, candidate_pairs, cosine_similarities = lsh_cosine_similarity(
    ratings_matrix, num_components=30, num_bands=10, rows_per_band=3, max_bucket_size=500, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with CS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 16/16 [00:00<00:00, 452.01it/s]


Number of Candidate pairs: 37718
Number of Pairs with CS > 0.73: 43
Pair with highest Cosine similarity: (14934, 33282) with a similarity of: 0.8254609256601646


In [61]:
# This is the best performing CS model and is thus used for the python file
filtered_pairs, candidate_pairs, cosine_similarities = lsh_cosine_similarity(
    ratings_matrix, num_components=20, num_bands=7, rows_per_band=3, max_bucket_size=500, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with CS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 12/12 [00:00<?, ?it/s]


Number of Candidate pairs: 28309
Number of Pairs with CS > 0.73: 72
Pair with highest Cosine similarity: (23703, 94471) with a similarity of: 0.8641629060441401


For some reason we find more similar pairs when lowering the signature length.

# 3: Discrete Cosine Similarity

In [46]:
def lsh_d_cosine_similarity(ratings_matrix, num_components=100, num_bands=33, rows_per_band=3, max_bucket_size=50, threshold=0.73):
    # Convert all non-zero elements of the ratings matrix to 1 for DCS
    ratings_matrix_dcs = ratings_matrix.copy()
    ratings_matrix_dcs.data = np.ones_like(ratings_matrix_dcs.data)

    ### Random Projections for LSH

    # Apply random projection to reduce dimensions
    rp = SparseRandomProjection(n_components=num_components, random_state=42)
    reduced_data = rp.fit_transform(ratings_matrix_dcs)

    ### LSH

    # Hashing the reduced data into buckets
    buckets = defaultdict(list)
    for user_id in tqdm(range(reduced_data.shape[0]), desc="Hashing users to buckets"):
        user_vector = reduced_data[user_id, :]
        for band_idx in range(num_bands):
            start_idx = band_idx * rows_per_band
            end_idx = start_idx + rows_per_band if band_idx < num_bands - 1 else num_components
            band = tuple(user_vector[start_idx:end_idx])
            bucket_key = hashlib.sha1(str(band).encode()).hexdigest()
            buckets[bucket_key].append(user_id)

    # Generating candidate pairs
    candidate_pairs = set()
    for bucket_users in tqdm(buckets.values(), desc="Generating candidate pairs"):
        if len(bucket_users) > 1 and len(bucket_users) <= max_bucket_size:
            for i in range(len(bucket_users)):
                for j in range(i + 1, len(bucket_users)):
                    candidate_pairs.add((bucket_users[i], bucket_users[j]))

    ### Calculate Discrete Cosine Similarities

    # Compute Cosine Similarity for Candidate Pairs
    discrete_cosine_similarities = {}
    for user1, user2 in candidate_pairs:
        # Convert to 1D arrays
        vector1 = reduced_data[user1, :].toarray().ravel()
        vector2 = reduced_data[user2, :].toarray().ravel()

        # Note: 1 - cosine distance is cosine similarity
        similarity = 1 - cosine(vector1, vector2)
        discrete_cosine_similarities[(user1, user2)] = similarity

    # Filter Pairs Based on Discrete Cosine Similarity Threshold
    filtered_pairs = {pair: sim for pair, sim in discrete_cosine_similarities.items() if sim > threshold}

    return filtered_pairs, candidate_pairs, discrete_cosine_similarities

In [47]:
filtered_pairs, candidate_pairs, discrete_cosine_similarities = lsh_d_cosine_similarity(
    ratings_matrix, num_components=100, num_bands=33, rows_per_band=3, max_bucket_size=50, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with DCS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(discrete_cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Discrete Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 44/44 [00:00<00:00, 43940.33it/s]


Number of Candidate pairs: 1376
Number of Pairs with DCS > 0.73: 0
Pair with highest Discrete Cosine similarity: (71373, 82597) with a similarity of: 0.6883566919601041


In [48]:
filtered_pairs, candidate_pairs, discrete_cosine_similarities = lsh_d_cosine_similarity(
    ratings_matrix, num_components=100, num_bands=33, rows_per_band=3, max_bucket_size=500, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with DCS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(discrete_cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Discrete Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 44/44 [00:00<00:00, 546.22it/s]


Number of Candidate pairs: 203415
Number of Pairs with DCS > 0.73: 6
Pair with highest Discrete Cosine similarity: (32556, 61876) with a similarity of: 0.7750268218467944


In [50]:
# This is the best performing DCS model and is thus used for the python file
filtered_pairs, candidate_pairs, discrete_cosine_similarities = lsh_d_cosine_similarity(
    ratings_matrix, num_components=80, num_bands=27, rows_per_band=3, max_bucket_size=500, threshold=0.73)
print("Number of Candidate pairs:", len(candidate_pairs))
print("Number of Pairs with DCS > 0.73:", len(filtered_pairs))
max_pair, max_similarity = max(discrete_cosine_similarities.items(), key=lambda item: item[1])
print("Pair with highest Discrete Cosine similarity:", max_pair, "with a similarity of:", max_similarity)

Generating candidate pairs: 100%|██████████| 38/38 [00:00<00:00, 757.25it/s]


Number of Candidate pairs: 160029
Number of Pairs with DCS > 0.73: 791
Pair with highest Discrete Cosine similarity: (33145, 58327) with a similarity of: 0.8244469218321544


We are able to find a decent amount of similar pairs with the DSC score.

### CMD prompts
python main.py -d "C:\Users\Niels\Desktop\ADM A2\user_movie_rating.npy" -s 42 -m js
python main.py -d "C:\Users\Niels\Desktop\ADM A2\user_movie_rating.npy" -s 42 -m cs
python main.py -d "C:\Users\Niels\Desktop\ADM A2\user_movie_rating.npy" -s 42 -m dcs
